# Video compression

According to the dataset paper, 

> However, for each target video in the test set, we randomly selected two clips out of three and applied augmentations that
approximate actual degradations seen in real-life video distributions. Specifically, these augmentations were
> 1. reduce the FPS of the video to 15
> 2. reduce the resolution of the video to 1/4 of its original size
> 3. reduce the overall encoding quality

In this notebook we look at how to compress videos using `ffmpeg`.  Based on the above list, a quick search returns several useful examples:

* General compression for the web 1: https://dev.to/benjaminblack/use-ffmpeg-to-compress-and-convert-videos-458l  
* General compression for the web 2: https://trac.ffmpeg.org/wiki/Encode/YouTube   
* Frame rate change: https://trac.ffmpeg.org/wiki/ChangingFrameRate  
* Scaling: https://trac.ffmpeg.org/wiki/Scaling  

The latest `ffmpeg` can be obtained at https://johnvansickle.com/ffmpeg/.  See the FAQ for how to install.

In [1]:
from IPython.display import HTML
from fastai.core import *
from fastai.vision import *
from kgl_deepfake.data import *

In [63]:
SOURCE = Path('../data/dfdc_train_part_10/')

In [64]:
fnames = get_files(SOURCE, extensions=['.mp4'])

In [68]:
fn = fnames[1390]
fn

PosixPath('../data/dfdc_train_part_10/rilulssouy.mp4')

In [69]:
HTML(html_vid(fn))

### File size

To conveniently check the file size of compressed videos:

In [59]:
def get_file_size(fpath):
    assert fpath.is_file()
    p = subprocess.run(f"du -hs {fpath}".split(), stdout=subprocess.PIPE)
    return p.stdout.decode().split('\t')[0]

In [60]:
Path.file_size = get_file_size

Original file size

In [70]:
fn.file_size()

'3.3M'

### Decorator to run command in terminal

In [10]:
def runnit(f):
    def _func(*args, **kwargs):
        command = f(*args, **kwargs)
        p = subprocess.run(command.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        return p
    return _func

### Are these videos HDR?

The following checks whether the video is HDR or not.

In [11]:
@runnit
def vid_colour_primaries(fpath):
    "Use `mediainfo` to get video's colour primaries."
    return f'''
    mediainfo f'{fpath}' --Inform="Video;%colour_primaries%"
    '''

def vid_is_HDR(fpath):
    "Is video HDR or not?"
    p = vid_colour_primaries(fpath)
    res = p.stdout.decode()
    return True if 'BT.2020' in res else False

In [13]:
vid_is_HDR(fn)

False

### Encoding

Each of the URLs above does something different: encoding, scaling, fps.  These are combined into a single function.  Note that probably not all combinations of input parameters will run, but the main parameters to adjust are:

* **crf** "The range of the CRF scale is 0–51, where 0 is lossless, 23 is the default, and 51 is worst quality possible. A lower value generally leads to higher quality, and a subjectively sane range is 17–28. Consider 17 or 18 to be visually lossless or nearly so; it should look the same or nearly the same as the input but it isn't technically lossless." > 23 recommended.
* **fps** "Changing frame rates requires the video to be re-encoded. Without setting appropriate output quality or bit rate, the video quality may be degraded. Please look at the respective encoding guides for the codec you've chosen."
* **height**/**wdith** Value between 0 and 1 to scale the original lengths.

To show a video's info after it's been compressed

In [40]:
def show_vid_info(f):
    def _f(*args, **kwargs):
        p = f(*args, **kwargs)
        fpath = Path(kwargs['fpath_to'])
        print(fpath.file_size())
        display(HTML(html_vid(fpath)))
    return _f

Function for donig the encoding using `ffmpeg`, with default arguments defined.

In [57]:
def _ffmpeg_defaults():
    return dict(video_encoder='libx264', video_bitrate='1.5M', preset='medium', 
                fps=30, width=1., height=1, crf=23, #17-28
                audio_encoder='aac', audio_bitrate='128k')

def _ffmpeg_opts():
    return dict(video_encoder='-c:v {video_encoder:s}', 
                video_bitrate='-b:v {video_bitrate:s}', 
                fps='-vf fps=fps={fps:d}', 
                scale='''-vf scale=iw*{width:.2f}:ih*{height:.2f}''', 
                crf='-crf {crf:d}',
                audio_bitrate='-b:a {audio_bitrate}', 
                audio_encoder='-c:a {audio_encoder}')

@show_vid_info
@runnit
def run_ffmpeg(fpath_from=None, fpath_to=None, **kwargs):
    opts_values = _ffmpeg_defaults()
    opts_values.update(kwargs)
    opts_str = ' '.join([s.format(**opts_values) for n, s in _ffmpeg_opts().items()])
    return f'ffmpeg -i {fpath_from} {opts_str} {fpath_to}'

Apply to the original:

In [71]:
f = Path(f'enc_{fn.name}')
if f.exists(): os.remove(f)
run_ffmpeg(fpath_from=fn, fpath_to=f, fps=15, width=.25, height=.25, crf=28)

208K


# -fin